# Decoder v3: Risk Score → Cat State

**목적**: MODEL 출력(risk_score)을 사용자가 보는 고양이 상태로 변환

**입력**: `model_out_v3.csv`  
**출력**: `state_out_v3.csv`

**Cat States**:
- 😺 STABLE: 안정
- 😴 SLEEP: 수면 패턴 변화
- 😾 LETHARGY: 무기력
- 😵 CHAOS: 혼란
- 🧳 TRAVEL: 여행/이동
- ⚫ NO_DATA: 데이터 부족

**CRITICAL (🚨)은 백엔드가 RDS에 직접 씀 - ML은 판정하지 않음**

# 0. 환경 설정 & 개요

## 0.1 라이브러리 임포트

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Optional, Tuple

print("✅ Libraries imported")

✅ Libraries imported


## 0.2 디코더 책임 범위

**DECODER의 역할**:
1. risk_score를 사용자 친화적 상태로 변환
2. Top-Z feature 기반 상태 해석 (SLEEP/LETHARGY/CHAOS)
3. Context 반영 (TRAVEL)
4. Notification level 결정

**하지 않는 것**:
- CRITICAL 판정 (백엔드가 RDS에 직접 씀)
- Risk 계산 (MODEL이 함)
- QC 판정 (FE가 함)

## 0.3 상수 정의

In [2]:
# === Risk Thresholds ===
# risk_score 분포 기반 임계값 (MODEL에서 p90/p95/p99)
RISK_LOW = 0.45      # p90 근처 (안정)
RISK_ALERT = 0.60    # p95 근처 (주의)
RISK_SEVERE = 0.70   # p99 근처 (경고)

# === State Priority ===
# 여러 조건 충족 시 우선순위 (높을수록 우선)
STATE_PRIORITY = {
    'NO_DATA': 100,
    'TRAVEL': 90,
    'CHAOS': 70,
    'LETHARGY': 60,
    'SLEEP': 50,
    'STABLE': 10,
}

# === Top-Z Feature Groups ===
SLEEP_FEATURES = ['night_ratio_z', 'overnight_gap_z']
LETHARGY_FEATURES = ['daily_event_cnt_z', 'UserAct_z', 'Screen_z']
CHAOS_FEATURES = ['hour_entropy_z', 'gap_max_z', 'gap_cnt_6h_z']

# === Z-score Threshold ===
# Top-Z 판정 기준 (절댓값)
Z_THRESHOLD = 2.0  # |z| >= 2.0 이면 유의미

# [v3] 그룹별 대표 Z 컬럼 (MODEL에서 export)
GROUP_REP_MAP = {
    "rhythm": "z_rhythm_rep",   # SLEEP 판정용
    "core":   "z_core_rep",     # LETHARGY 판정용
    "gap":    "z_gap_rep",      # CHAOS 판정용
    "session":"z_session_rep",  # 보조
}
GROUP_Z_THRESHOLD = 1.5  # 그룹 대표값 기준 (개별 피처보다 낮게)

# === Cold-Start Stages ===
STAGE_NOTIFY_MAP = {
    'ONBOARD': 'NONE',      # 1-4일: 알림 없음
    'WARMUP': 'NONE',       # 5-8일: 알림 없음
    'SEMI_READY': 'LOW',    # 9-15일: 낮은 알림만
    'READY': 'NORMAL',      # 16일~: 정상 알림
}

print("✅ Constants defined")
print(f"Risk thresholds: LOW={RISK_LOW}, ALERT={RISK_ALERT}, SEV={RISK_SEVERE}")

✅ Constants defined
Risk thresholds: LOW=0.45, ALERT=0.6, SEV=0.7


# 1. 데이터 로드 & 스키마 검증

## 1.1 model_out_v3.csv 로드

In [3]:
MODEL_PATH = Path("../artifacts/model/model_out_v3.csv")

if not MODEL_PATH.exists():
    raise FileNotFoundError(f"Model output not found: {MODEL_PATH}")

df = pd.read_csv(MODEL_PATH)
df['date'] = pd.to_datetime(df['date'])

print(f"✅ Loaded: {MODEL_PATH}")
print(f"Rows: {len(df):,} | Users: {df['uuid'].nunique()}")
print(f"Columns: {len(df.columns)}")
df.head(3)

✅ Loaded: ..\artifacts\model\model_out_v3.csv
Rows: 9,057 | Users: 342
Columns: 44


,uuid,date,day_idx,cold_stage,early_ready,early_risk,final_risk,quality_state,context_mode,baseline_ready,...,gap_cnt_2h_z,gap_cnt_6h_z,gap_long_ratio_z,overnight_gap_z,daily_event_cnt_drift_z,night_ratio_drift_z,gap_max_drift_z,z_core_rep,z_rhythm_rep,z_gap_rep
0,u072142,2016-06-19,1,ONBOARD,False,NaN,NaN,GOOD,NORMAL,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,u072142,2016-06-20,2,ONBOARD,False,NaN,NaN,GOOD,NORMAL,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,u072142,2016-06-21,3,ONBOARD,False,NaN,NaN,GOOD,NORMAL,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1.2 필수 컬럼 존재 체크

In [4]:
REQUIRED_COLS = [
    "uuid", "date",
    # risk
    "risk_score",            # 최소
    # quality/context
    "quality_state",
    "cold_stage",
    "travel_flag",
]

missing = [c for c in REQUIRED_COLS if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns: {missing}")

# ---- optional columns: 없으면 default 생성 (운영 안정성)
OPTIONAL_DEFAULTS = {
    "final_risk": np.nan,        # 있으면 risk_used 우선
    "early_risk": np.nan,
    "baseline_ready": False,
    "early_ready": False,
    "has_activity": False,

    # context flags (있으면 downshift에 사용)
    "partial_flag": False,
    "tz_flag": False,

    # recovery flags (MODEL에서 생성)
    "stable_hold_flag": False,
    "drift_flag": False,
    "drift_top_feature": "",
}
for c, v in OPTIONAL_DEFAULTS.items():
    if c not in df.columns:
        df[c] = v

# Z-score 컬럼 확인
z_cols = [c for c in df.columns if c.endswith("_z") and not c.endswith("_z_early")]
print("✅ Schema OK")
print(f"Z-score columns: {len(z_cols)}")
print(f"Examples: {z_cols[:5]}")

✅ Schema OK
Z-score columns: 15
Examples: ['daily_event_cnt_z', 'Screen_z', 'Notif_z', 'UserAct_z', 'night_ratio_z']


## 1.3 컬럼 타입 표준화

In [5]:
df = df.copy()

# Boolean 플래그(없으면 이미 default 생성됨)
BOOL_COLS = ["travel_flag", "partial_flag", "tz_flag", "baseline_ready", "early_ready", "has_activity", "stable_hold_flag", "drift_flag"]
for c in BOOL_COLS:
    if c in df.columns:
        df[c] = df[c].astype("boolean").fillna(False)

# Numeric
for c in ["risk_score", "final_risk", "early_risk"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# String
for c in ["quality_state", "context_mode", "cold_stage"]:
    if c in df.columns:
        df[c] = df[c].astype("string")

# ✅ 핵심: risk_used 통일 (final_risk 우선, 없으면 risk_score)
df["risk_used"] = df["final_risk"]
df.loc[df["risk_used"].isna(), "risk_used"] = df.loc[df["risk_used"].isna(), "risk_score"]

print("✅ Type normalization done")
print(f"risk_used valid: {df['risk_used'].notna().sum():,} / {len(df):,}")
print(f"quality_state dist: {df['quality_state'].value_counts(dropna=False).to_dict()}")


✅ Type normalization done
risk_used valid: 7,352 / 9,057
quality_state dist: {'GOOD': 8702, 'LOW_CONF': 355}


# 2. Data Quality Gate

## 2.1 NO_DATA 판정

In [6]:
df = df.copy()

# (1) 품질 실패: FE/QC가 LOW_CONF면 무조건 분석 불가
df["quality_fail"] = (df["quality_state"] == "LOW_CONF")

# (2) 분석 가능(최소안):
# - quality_fail 아님
# - risk_used 존재
# - baseline_ready or early_ready (둘 다 없으면 모델 점수 신뢰도 부족)
df["analysis_ready"] = (
    (~df["quality_fail"]) &
    (df["risk_used"].notna()) &
    (df["baseline_ready"] | df["early_ready"])
)

# (3) 최종 NO_DATA: 분석 불가면 NO_DATA
df["is_no_data"] = ~df["analysis_ready"]

print("=== NO_DATA Gate (v3) ===")
print(f"NO_DATA: {df['is_no_data'].sum():,} / {len(df):,} ({df['is_no_data'].mean():.1%})")
print(f"  - quality_fail(LOW_CONF): {df['quality_fail'].sum():,}")
print(f"  - risk_used NaN: {df['risk_used'].isna().sum():,}")
print(f"  - baseline_ready False & early_ready False: {((~df['baseline_ready']) & (~df['early_ready'])).sum():,}")

=== NO_DATA Gate (v3) ===
NO_DATA: 1,705 / 9,057 (18.8%)
  - quality_fail(LOW_CONF): 355
  - risk_used NaN: 1,705
  - baseline_ready False & early_ready False: 1,468


## 2.2 분석 가능 마스크

In [7]:
df["analysis_ok"] = df["analysis_ready"]
print(f"✅ Analysis OK: {df['analysis_ok'].sum():,} / {len(df):,} ({df['analysis_ok'].mean():.1%})")

✅ Analysis OK: 7,352 / 9,057 (81.2%)


# 3. Risk Band & Base State

## 3.1 Risk Band 계산

In [8]:
def get_risk_band(risk: float) -> str:
    """risk_used를 band로 변환"""
    if pd.isna(risk):
        return "UNKNOWN"
    if risk < RISK_LOW:
        return "SAFE"
    elif risk < RISK_ALERT:
        return "WATCH"
    elif risk < RISK_SEVERE:
        return "ALERT"
    else:
        return "SEVERE"

df["risk_band"] = df["risk_used"].apply(get_risk_band)

print("=== Risk Band Distribution ===")
print(df.loc[df["analysis_ok"], "risk_band"].value_counts())
print("\nRisk score stats (analysis_ok):")
print(df.loc[df["analysis_ok"], "risk_used"].describe())

=== Risk Band Distribution ===
risk_band
SAFE      6102
WATCH      696
SEVERE     334
ALERT      220
Name: count, dtype: int64

Risk score stats (analysis_ok):
count    7352.000000
mean        0.361768
std         0.135779
min         0.058131
25%         0.277076
50%         0.329585
75%         0.404437
max         0.966379
Name: risk_used, dtype: float64


## 3.2 Base State 매핑

In [9]:
# [v3] base_state 제거 — determine_cat_state에서 pattern_state만 사용하므로 불필요
# 상태 결정은 pattern_state(그룹별 Z 기반)가 담당
print("✅ base_state 제거 (pattern_state로 통합)")

✅ base_state 제거 (pattern_state로 통합)


# 4. Context Override

## 4.1 TRAVEL 처리

In [10]:
# TRAVEL flag가 있으면 무조건 TRAVEL
travel_mask = df['travel_flag'].fillna(False)

df['is_travel'] = travel_mask

print(f"=== TRAVEL Override ===")
print(f"TRAVEL: {travel_mask.sum():,} / {len(df):,} ({travel_mask.mean():.1%})")

=== TRAVEL Override ===
TRAVEL: 0 / 9,057 (0.0%)


# 5. Analysis-Level State (Top-Z)

## 5.1 Top-Z Feature 추출

In [11]:
# 5. Analysis-Level State (Top-Z)
# 5.1 Top-Z Feature 추출 (analysis_ok에서만)
z_cols = [c for c in df.columns if c.endswith("_z") and not c.endswith("_z_early")]

df["top_z_feature"] = "none"
df["top_z_value"] = 0.0

def get_top_z_feature(row: pd.Series, z_cols: List[str]) -> Tuple[str, float]:
    zvals = row[z_cols]
    zvals = zvals[pd.notna(zvals)]
    if len(zvals) == 0:
        return ("none", 0.0)
    # abs 최대
    top_col = (zvals.abs()).idxmax()
    return (top_col, float(zvals[top_col]))

mask = df["analysis_ok"]
topz_results = df.loc[mask].apply(lambda r: get_top_z_feature(r, z_cols), axis=1)
df.loc[mask, "top_z_feature"] = topz_results.apply(lambda x: x[0]).astype("string")
df.loc[mask, "top_z_value"] = topz_results.apply(lambda x: x[1]).astype(float)

print("=== Top-Z Feature Extraction ===")
print("Top-Z features (top 10):")
print(df.loc[df["analysis_ok"], "top_z_feature"].value_counts().head(10))


=== Top-Z Feature Extraction ===
Top-Z features (top 10):
top_z_feature
Notif_z                    1001
daily_event_cnt_drift_z     916
UserAct_z                   811
Screen_z                    765
gap_max_drift_z             757
overnight_gap_z             613
hour_entropy_z              519
gap_max_z                   411
gap_long_ratio_z            354
night_ratio_z               276
Name: count, dtype: int64


## 5.2 패턴별 상태 판정

In [12]:
def decode_pattern_state(row: pd.Series) -> str:
    """
    [v3] 그룹별 대표 Z + Top-Z feature 기반 상태 판정
    
    1차: 그룹 대표값(z_rhythm_rep, z_core_rep, z_gap_rep)으로 어떤 영역이 이상한지 판단
    2차: Top-Z feature로 세부 확인 (fallback)
    """
    top_feature = row["top_z_feature"]
    top_value   = row["top_z_value"]
    
    # 그룹 대표값 읽기 (없으면 0)
    rhythm_rep = float(row.get("z_rhythm_rep", 0) or 0)
    core_rep   = float(row.get("z_core_rep", 0) or 0)
    gap_rep    = float(row.get("z_gap_rep", 0) or 0)
    session_rep= float(row.get("z_session_rep", 0) or 0)
    
    # 그룹 대표값 기반 후보 수집 (threshold 이상인 그룹들)
    candidates = {}
    if rhythm_rep >= GROUP_Z_THRESHOLD:
        candidates["SLEEP"] = rhythm_rep
    if core_rep >= GROUP_Z_THRESHOLD:
        # LETHARGY는 활동 '감소'일 때만 → top_z가 음수인지 확인
        if top_feature in LETHARGY_FEATURES and top_value < 0:
            candidates["LETHARGY"] = core_rep
        elif top_feature not in LETHARGY_FEATURES:
            # core가 높지만 top_z가 lethargy 피처가 아닌 경우 → 활동 급증일 수 있음
            pass
        else:
            candidates["LETHARGY"] = core_rep
    if gap_rep >= GROUP_Z_THRESHOLD:
        candidates["CHAOS"] = gap_rep
    
    # 후보가 있으면 가장 높은 그룹으로
    if candidates:
        return max(candidates, key=candidates.get)
    
    # fallback: 그룹 대표값이 없으면 기존 Top-Z 로직
    if abs(top_value) < Z_THRESHOLD:
        return "STABLE"
    
    if top_feature in SLEEP_FEATURES:
        return "SLEEP"
    if top_feature in LETHARGY_FEATURES:
        return "LETHARGY" if top_value < 0 else "STABLE"
    if top_feature in CHAOS_FEATURES:
        return "CHAOS"
    
    return "STABLE"


df["pattern_state"] = "UNKNOWN"
df.loc[df["analysis_ok"], "pattern_state"] = df.loc[df["analysis_ok"]].apply(decode_pattern_state, axis=1)

print("=== Pattern State (v3: group-based) ===")
print(df.loc[df["analysis_ok"], "pattern_state"].value_counts())


=== Pattern State (v3: group-based) ===
pattern_state
STABLE      4008
CHAOS       1322
LETHARGY    1237
SLEEP        785
Name: count, dtype: int64


## 5.3 최종 Cat State 결정

In [13]:
def determine_cat_state(row: pd.Series) -> str:
    """
    우선순위 기반 최종 상태 결정
    
    Priority:
    1. NO_DATA (데이터 품질)
    2. TRAVEL (컨텍스트)
    3. Pattern state (분석)
    """
    # 1) NO_DATA
    if row['is_no_data']:
        return 'NO_DATA'
    
    # 2) TRAVEL
    if row['is_travel']:
        return 'TRAVEL'
    
    # 3) Pattern state
    return row['pattern_state']

df['cat_state'] = df.apply(determine_cat_state, axis=1)

print("=== Final Cat State ===")
print(df['cat_state'].value_counts())
print(f"\nDistribution:")
for state in ['STABLE', 'SLEEP', 'LETHARGY', 'CHAOS', 'TRAVEL', 'NO_DATA']:
    count = (df['cat_state'] == state).sum()
    pct = count / len(df) * 100
    print(f"  {state:12s}: {count:5,} ({pct:5.1f}%)")

=== Final Cat State ===
cat_state
STABLE      4008
NO_DATA     1705
CHAOS       1322
LETHARGY    1237
SLEEP        785
Name: count, dtype: int64

Distribution:
  STABLE      : 4,008 ( 44.3%)
  SLEEP       :   785 (  8.7%)
  LETHARGY    : 1,237 ( 13.7%)
  CHAOS       : 1,322 ( 14.6%)
  TRAVEL      :     0 (  0.0%)
  NO_DATA     : 1,705 ( 18.8%)


# 6. Notification Level 결정

## 6.1 Base Notify Level

In [14]:
def get_base_notify(risk_band: str) -> str:
    """
    Risk band 기반 기본 알림 레벨
    """
    if risk_band == 'SAFE':
        return 'NONE'
    elif risk_band == 'WATCH':
        return 'NONE'
    elif risk_band == 'ALERT':
        return 'LOW'
    elif risk_band == 'SEVERE':
        return 'HIGH'
    else:
        return 'NONE'

df['base_notify'] = df['risk_band'].apply(get_base_notify)

print("=== Base Notify Level ===")
print(df.loc[df['analysis_ok'], 'base_notify'].value_counts())

=== Base Notify Level ===
base_notify
NONE    6798
HIGH     334
LOW      220
Name: count, dtype: int64


## 6.2 Context Downshift

In [15]:
def apply_context_downshift(row: pd.Series) -> str:
    """
    Context에 따라 알림 레벨 낮춤 (v3)
    - TRAVEL/PARTIAL/TZ → NONE (MVP 정책)
    """
    base = row["base_notify"]

    if row["is_travel"]:
        return "NONE"

    # flag 기반 downshift (문자열 contains 제거)
    if bool(row.get("partial_flag", False)) or bool(row.get("tz_flag", False)):
        return "NONE"

    return base

df["notify_after_context"] = df.apply(apply_context_downshift, axis=1)

print("=== After Context Downshift ===")
print(df["notify_after_context"].value_counts())


=== After Context Downshift ===
notify_after_context
NONE    8503
HIGH     334
LOW      220
Name: count, dtype: int64


## 6.3 최종 Notify Level

In [16]:
def determine_notify_level(row: pd.Series) -> str:
    """
    최종 알림 레벨 결정 (cold_start 적용 전)
    - NO_DATA/TRAVEL은 NONE
    - STABLE은 NONE (MVP: 안정은 알림 없음)
    - [v3] stable_hold_flag: 최근 심각했다가 안정으로 내려온 경우 → LOW 유지 (모니터링 계속)
    - 그 외는 context 반영 notify
    """
    if row["cat_state"] in ["NO_DATA", "TRAVEL"]:
        return "NONE"
    if row["cat_state"] == "STABLE":
        # [v3] 최근 2주 내 심각했던 이력이 있으면 바로 알림을 끄지 않음
        if bool(row.get("stable_hold_flag", False)):
            return "LOW"
        # [v3] baseline drift: 서서히 나빠지고 있으면 Z-score는 낮아도 알림
        if bool(row.get("drift_flag", False)):
            return "LOW"
        return "NONE"
    return row["notify_after_context"]

df["notify_level"] = df.apply(determine_notify_level, axis=1)

print("=== Final Notify Level (pre-cold) ===")
print(df["notify_level"].value_counts())
print(f"\nNotify rate: {(df['notify_level'] != 'NONE').mean():.1%}")


=== Final Notify Level (pre-cold) ===
notify_level
NONE    8120
LOW      898
HIGH      39
Name: count, dtype: int64

Notify rate: 10.3%


# 7. Cold-Start Policy 적용

## 7.1 Stage별 알림 조정

In [17]:
def apply_cold_start_policy(row: pd.Series) -> str:
    """
    Cold-start stage에 따라 알림 억제
    
    ONBOARD/WARMUP → NONE
    SEMI_READY → HIGH만 LOW로
    READY → 그대로
    """
    stage = row['cold_stage']
    notify = row['notify_level']
    
    if pd.isna(stage):
        return notify
    
    stage = str(stage).upper()
    
    if stage in ['ONBOARD', 'WARMUP']:
        return 'NONE'
    elif stage == 'SEMI_READY':
        if notify == 'HIGH':
            return 'LOW'
        return notify
    else:  # READY
        return notify

df['notify_final'] = df.apply(apply_cold_start_policy, axis=1)

print("=== After Cold-Start Policy ===")
print(df['notify_final'].value_counts())
print(f"\nBy stage:")
for stage in ['ONBOARD', 'WARMUP', 'SEMI_READY', 'READY']:
    stage_df = df[df['cold_stage'] == stage]
    if len(stage_df) > 0:
        notify_dist = stage_df['notify_final'].value_counts().to_dict()
        print(f"  {stage:12s}: {notify_dist}")

=== After Cold-Start Policy ===
notify_final
NONE    8120
LOW      933
HIGH       4
Name: count, dtype: int64

By stage:
  ONBOARD     : {'NONE': 1346}
  WARMUP      : {'NONE': 1339}
  SEMI_READY  : {'NONE': 1891, 'LOW': 395}
  READY       : {'NONE': 3544, 'LOW': 538, 'HIGH': 4}


# 8. Output Assembly & Validation

## 8.1 Decoder Quality 판정

In [18]:
def get_decoder_quality(row: pd.Series) -> str:
    """
    Decoder 처리 품질 표시
    """
    if row['cat_state'] == 'NO_DATA':
        if row['quality_state'] == 'LOW_CONF':
            return 'LOW_CONF'
        else:
            return 'NO_DATA'
    else:
        return 'OK'

df['decoder_quality'] = df.apply(get_decoder_quality, axis=1)

print("=== Decoder Quality ===")
print(df['decoder_quality'].value_counts())

=== Decoder Quality ===
decoder_quality
OK          7352
NO_DATA     1350
LOW_CONF     355
Name: count, dtype: int64


## 8.2 최종 Output 생성

In [19]:
# 출력 컬럼 정의
OUTPUT_COLS = [
    "uuid",
    "date",
    "cat_state",
    "notify_level",
    "decoder_quality",
    # 디버그용
    "risk_used",
    "risk_score",
    "final_risk",
    "risk_band",
    "top_z_feature",
    "top_z_value",
]

# notify_level은 notify_final 사용
output_df = df.copy()
output_df['notify_level'] = output_df['notify_final']

# 날짜 형식 변환
output_df['date'] = pd.to_datetime(output_df['date']).dt.strftime('%Y-%m-%d')

# 최종 선택
final_output = output_df[OUTPUT_COLS].copy()

print("=== Output Shape ===")
print(f"Rows: {len(final_output):,}")
print(f"Columns: {len(final_output.columns)}")
print(f"\nFirst 5 rows:")
final_output.head()

=== Output Shape ===
Rows: 9,057
Columns: 11

First 5 rows:


,uuid,date,cat_state,notify_level,decoder_quality,risk_used,risk_score,final_risk,risk_band,top_z_feature,top_z_value
0,u072142,2016-06-19,NO_DATA,NONE,NO_DATA,NaN,NaN,NaN,UNKNOWN,none,0.000000
1,u072142,2016-06-20,NO_DATA,NONE,NO_DATA,NaN,NaN,NaN,UNKNOWN,none,0.000000
2,u072142,2016-06-21,NO_DATA,NONE,NO_DATA,NaN,NaN,NaN,UNKNOWN,none,0.000000
3,u072142,2016-06-22,NO_DATA,NONE,NO_DATA,NaN,NaN,NaN,UNKNOWN,none,0.000000
4,u072142,2016-06-23,STABLE,NONE,OK,0.610378,NaN,0.610378,ALERT,gap_max_drift_z,1.505851


## 8.3 중복키 체크

In [20]:
# uuid + date 중복 체크
duplicates = final_output.duplicated(subset=['uuid', 'date'], keep=False)

if duplicates.any():
    print(f"⚠️ WARNING: {duplicates.sum()} duplicate rows found")
    print(final_output[duplicates].head())
else:
    print("✅ No duplicates (uuid + date unique)")

✅ No duplicates (uuid + date unique)


## 8.4 State 분포 리포트

In [21]:
print("="*50)
print("📊 DECODER OUTPUT SUMMARY")
print("="*50)

print(f"\n[Cat State Distribution]")
state_dist = final_output['cat_state'].value_counts()
for state, count in state_dist.items():
    pct = count / len(final_output) * 100
    emoji = {'STABLE': '😺', 'SLEEP': '😴', 'LETHARGY': '😾', 
             'CHAOS': '😵', 'TRAVEL': '🧳', 'NO_DATA': '⚫'}.get(state, '')
    print(f"  {emoji} {state:12s}: {count:5,} ({pct:5.1f}%)")

print(f"\n[Notify Level Distribution]")
notify_dist = final_output['notify_level'].value_counts()
for level, count in notify_dist.items():
    pct = count / len(final_output) * 100
    print(f"  {level:8s}: {count:5,} ({pct:5.1f}%)")

print(f"\n[Decoder Quality]")
quality_dist = final_output['decoder_quality'].value_counts()
for qual, count in quality_dist.items():
    pct = count / len(final_output) * 100
    print(f"  {qual:12s}: {count:5,} ({pct:5.1f}%)")

print(f"\n[Risk Band (analysis_ok only)]")
risk_ok = final_output[final_output['decoder_quality'] == 'OK']
risk_dist = risk_ok['risk_band'].value_counts()
for band, count in risk_dist.items():
    pct = count / len(risk_ok) * 100 if len(risk_ok) > 0 else 0
    print(f"  {band:8s}: {count:5,} ({pct:5.1f}%)")

print("="*50)

📊 DECODER OUTPUT SUMMARY

[Cat State Distribution]
  😺 STABLE      : 4,008 ( 44.3%)
  ⚫ NO_DATA     : 1,705 ( 18.8%)
  😵 CHAOS       : 1,322 ( 14.6%)
  😾 LETHARGY    : 1,237 ( 13.7%)
  😴 SLEEP       :   785 (  8.7%)

[Notify Level Distribution]
  NONE    : 8,120 ( 89.7%)
  LOW     :   933 ( 10.3%)
  HIGH    :     4 (  0.0%)

[Decoder Quality]
  OK          : 7,352 ( 81.2%)
  NO_DATA     : 1,350 ( 14.9%)
  LOW_CONF    :   355 (  3.9%)

[Risk Band (analysis_ok only)]
  SAFE    : 6,102 ( 83.0%)
  WATCH   :   696 (  9.5%)
  SEVERE  :   334 (  4.5%)
  ALERT   :   220 (  3.0%)


## 8.5 파일 저장

In [22]:
OUT_DIR = Path("../artifacts/decoder")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_PATH = OUT_DIR / "state_out_v3.csv"

final_output.to_csv(OUT_PATH, index=False)

print("="*50)
print("✅ DECODER OUTPUT SAVED")
print("="*50)
print(f"Path: {OUT_PATH.resolve()}")
print(f"Rows: {len(final_output):,}")
print(f"Users: {final_output['uuid'].nunique()}")
print("="*50)

✅ DECODER OUTPUT SAVED
Path: C:\Users\DS3\Desktop\amz\artifacts\decoder\state_out_v3.csv
Rows: 9,057
Users: 342


# 9. Decoder Quality Report

## 9.1 상태 전환 매트릭스

In [23]:
# 날짜순 정렬
transition_df = final_output.sort_values(['uuid', 'date']).copy()

# 이전 상태
transition_df['prev_state'] = transition_df.groupby('uuid')['cat_state'].shift(1)

# 전환만 추출
transitions = transition_df[transition_df['prev_state'].notna()].copy()

if len(transitions) > 0:
    print("=== State Transition Matrix ===")
    transition_matrix = pd.crosstab(
        transitions['prev_state'], 
        transitions['cat_state'],
        margins=True
    )
    print(transition_matrix)
    
    # 전환 빈도 (상위 10개)
    print("\n=== Top 10 Transitions ===")
    trans_counts = transitions.groupby(['prev_state', 'cat_state']).size().sort_values(ascending=False)
    print(trans_counts.head(10))
else:
    print("No transitions found (single-day users)")

=== State Transition Matrix ===
cat_state   CHAOS  LETHARGY  NO_DATA  SLEEP  STABLE   All
prev_state                                               
CHAOS         307       210       35    171     530  1253
LETHARGY      245       342       13    156     439  1195
NO_DATA        36        11     1241     17     377  1682
SLEEP         193       120       25    114     267   719
STABLE        541       554       49    327    2395  3866
All          1322      1237     1363    785    4008  8715

=== Top 10 Transitions ===
prev_state  cat_state
STABLE      STABLE       2395
NO_DATA     NO_DATA      1241
STABLE      LETHARGY      554
            CHAOS         541
CHAOS       STABLE        530
LETHARGY    STABLE        439
NO_DATA     STABLE        377
LETHARGY    LETHARGY      342
STABLE      SLEEP         327
CHAOS       CHAOS         307
dtype: int64


## 9.2 Top-Z Feature 통계

In [24]:
# 분석 가능한 날만
analysis_df = final_output[final_output['decoder_quality'] == 'OK'].copy()

if len(analysis_df) > 0:
    print("=== Top-Z Feature by Cat State ===")
    for state in ['SLEEP', 'LETHARGY', 'CHAOS', 'STABLE']:
        state_df = analysis_df[analysis_df['cat_state'] == state]
        if len(state_df) > 0:
            print(f"\n{state}:")
            top_features = state_df['top_z_feature'].value_counts().head(5)
            print(top_features)
else:
    print("No analysis_ok data")

=== Top-Z Feature by Cat State ===

SLEEP:
top_z_feature
hour_entropy_z             333
night_ratio_z              191
Notif_z                    134
overnight_gap_z             46
daily_event_cnt_drift_z     44
Name: count, dtype: int64

LETHARGY:
top_z_feature
UserAct_z            594
Screen_z             477
daily_event_cnt_z    166
Name: count, dtype: int64

CHAOS:
top_z_feature
overnight_gap_z            349
Notif_z                    282
gap_max_z                  273
gap_long_ratio_z           182
daily_event_cnt_drift_z     82
Name: count, dtype: int64

STABLE:
top_z_feature
daily_event_cnt_drift_z    790
gap_max_drift_z            743
Notif_z                    585
Screen_z                   265
overnight_gap_z            218
Name: count, dtype: int64


## 9.3 유저별 상태 분포

In [25]:
# 유저별 가장 많은 상태
user_mode_state = final_output.groupby('uuid')['cat_state'].agg(lambda x: x.mode()[0] if len(x.mode()) > 0 else 'UNKNOWN')

print("=== User Mode State Distribution ===")
print(user_mode_state.value_counts())

# 알림 받은 유저 비율
users_with_notify = final_output[final_output['notify_level'] != 'NONE']['uuid'].nunique()
total_users = final_output['uuid'].nunique()

print(f"\n=== Notification Coverage ===")
print(f"Users with notifications: {users_with_notify} / {total_users} ({users_with_notify/total_users:.1%})")

=== User Mode State Distribution ===
cat_state
STABLE      307
NO_DATA      25
CHAOS         6
LETHARGY      3
SLEEP         1
Name: count, dtype: int64

=== Notification Coverage ===
Users with notifications: 229 / 342 (67.0%)
